# Explaination of Lab

## 00-a Clone course repository
- First of all clone the LAB repository in your own github 

## 00-b Gitignore
- After creating the lab repositry in your github create the gitignore file and put following to ignore them during your lab project
- .wal
- .DS Store
- .dbeaver
- .project

## 00-c Changing folder name

- To perform this lab , we have alot of data in different schemas and we want to change the name of those schemas.
- For example from Datum 2024-08-13_2024-09-10 AIgineer and those schemas we have in lab in backend folder - raw data.
- We want to keep the schema name like Datum and rest remove it and copy them in clean data folder. 

### To perform this action we did some coding.

- Frist to all mention the path for raw data folder with the help of Pathlib and same for clean data folder 
- Create the clean folder so copy the renamed schemas  and this copying the schemas is happended with help of #copytree
- With the help of below block of code , change the raw schema into new schema name. 
```python
from pathlib import Path
from shutil import copytree, rmtree

raw_data_path = Path(__file__).parent / "raw_data"
cleaned_data_path = Path(__file__).parent / "cleaned_data"

if cleaned_data_path.is_dir():
    rmtree(cleaned_data_path)

cleaned_data_path.mkdir(parents=True, exist_ok=True)

for folder in raw_data_path.iterdir():
    new_name = folder.name.split()[0]
    copytree(folder, cleaned_data_path / new_name)
```


# 00-d Ingesting the CSV file in database with help of Python

- We have two CSV files in data folder and find where is the CSV file is with help of DATA_PATH.
- With the hep of setup_schema funtions set a schema with the name of supplier_invoice.
- Clean up the file name , create a table for those invoices and import the CSV data into the data.  

```python
def ingest_csv_data_to_duckdb():
    """ingesting the cleaned csv data to duckdb"""
    translation_table = str.maketrans({"å": "a", "ä": "a", "ö": "o"})

    for directory_path in CLEANED_DATA_PATH.glob("*"):
        schema_name = directory_path.name.lower().translate(translation_table)
        for csv_path in directory_path.glob("*"):
            table_name = csv_path.name.lower().split(".")[0].translate(translation_table)
```

## 00-e EDA on different schema

- Exploratry data analysist to better understand the data
- We explor different schema for exmaple datum and explor the two table in that schema like table and total.
- We fix the format of date column into Year-Month-Day.
- We have also join them to explor the data also see them in another angel. 

```sql
SELECT
    STRFTIME('%y-%m-%d', GEO_TOT.Datum) AS Datum,
    ANY_VALUE(GEO_TAB.Visningar) AS Visningar,
    ANY_VALUE(GEO_TAB."Visningstid (timmar)") AS "Visningstid (timmar)",
    GEO_TAB.Geografi
FROM  
    geografi.tabelldata AS geo_tab
CROSS JOIN
    geografi.totalt AS geo_tot
GROUP BY
    GEO_TAB.Geografi,
    STRFTIME('%y-%m-%d', GEO_TOT.Datum);
```


- Same way i did the EDA for geografi schema , run the different table in the schema to have good understand
- Did the Cross Join the two table in geografi schema and Group by different column to see on which Date , Geografi , Visningar , Visningar (timmar) and Genomsnittlig visningslängd 

```sql
SELECT * FROM  geografi.diagramdata d ;
   
SELECT  * FROM  geografi.tabelldata t ;
  
SELECT * FROM  geografi.totalt t ;

SELECT
    STRFTIME('%y-%m-%d', GEO_TOT.Datum) AS Datum,
    GEO_TAB.* 
FROM  
    geografi.tabelldata AS geo_tab
CROSS JOIN
    geografi.totalt  AS geo_tot
GROUP BY
    Datum, GEO_TAB.Geografi , GEO_TAB.Visningar , GEO_TAB."Visningstid (timmar)" , GEO_TAB."Genomsnittlig visningslängd" ;


